# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD2030 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

# Clase 4: Webscraping.

El proceso de webscrapping en su expresión mas básica consiste en la extracción de datos desde el texto plano que estructura la página web (HTML). Requests nos brinda herramientas para la extracción de la página, y a continuación podemos usar otras librerías como BeautifulSoup para la extracción de elementos o información contenida en el texto plano.

Este Notebook contiene los códigos de ejemplo presentados en la videoclase titulada *"Raspado de páginas web - webscraping"*.


In [1]:
#Importación de librerías
import requests
from bs4 import BeautifulSoup 
import pandas as pd

# 2.Código HTML

Veamos primero un ejemplo simple de código HTML para comprender su estructura general.

Se sugiere descargar el archivo HTML de ejemplo y abrirlo con un navegador (Firefox, Chrome, etc.), de manera de visualizarlo como pagina web.

In [2]:
#archivo HTML de ejemplo disponible en Github
html_text=requests.get('https://raw.githubusercontent.com/paguirre-uc/mds3020_2022/main/semana2/ejemplo.html').content

#alternativa si descargan el archivo HTML de ejemplo a su directorio local.
#html_text=open('ejemplo.html','r').read()

html_text

b'<html>\n    <head>\n        <title>Clase 4: Web Scrapping</title>\n    </head>\n    <body>\n        <h1>C\xc3\xb3digo HTML</h1>\n        <p> Este es un ejemplo para comprender la estructura de un documento HTML.\n        \n        Ac\xc3\xa1 vemos un ejemplo de lista con 3 elementos:\n          <ul id=\'list\' class=\'pasos\'>\n            <li> Elemento 1: 100</li>\n            <li> Elemento 2: 200 </li>\n            <li> Elemento 3: 300</li>\n          </ul>\n        </p>\n\n        <p>Tambi\xc3\xa9n podemos crear una tabla, por ejemplo con la informaci\xc3\xb3n de los cursos (sigla, cr\xc3\xa9ditos y n\xc3\xbamero de estudiantes) de un programa de estudios:\n        <p></p>\n          <table>\n           <tr>\n             <th>Curso</th>\n             <th>Cr\xc3\xa9ditos</th>\n             <th>N Estudiantes</th>\n           </tr>\n           <tr>\n             <td style="text-align: center; vertical-align: middle;">IMT2100</td>\n             <td style="text-align: center; vertical-

In [3]:
from bs4 import BeautifulSoup 

#Creamos una "sopa" a partir del códiog HTML
soup = BeautifulSoup(html_text)

In [4]:
#accedemos a la tabla de datos. Esto devuelve la primera tabla de datos del código.
soup.table

<table>
<tr>
<th>Curso</th>
<th>Créditos</th>
<th>N Estudiantes</th>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">IMT2100</td>
<td style="text-align: center; vertical-align: middle;">10</td>
<td style="text-align: center; vertical-align: middle;">42</td>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">IMT2200</td>
<td style="text-align: center; vertical-align: middle;">10</td>
<td style="text-align: center; vertical-align: middle;">25</td>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">OFG1100</td>
<td style="text-align: center; vertical-align: middle;">5</td>
<td style="text-align: center; vertical-align: middle;">35</td>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">OFG1200</td>
<td style="text-align: center; vertical-align: middle;">5</td>
<td style="text-align: center; vertical-align: middle;">28</td>
</tr>
</table>

In [5]:
#Otra forma de extraer la tabla de datos.
table=soup.find('table')
print(table)

<table>
<tr>
<th>Curso</th>
<th>Créditos</th>
<th>N Estudiantes</th>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">IMT2100</td>
<td style="text-align: center; vertical-align: middle;">10</td>
<td style="text-align: center; vertical-align: middle;">42</td>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">IMT2200</td>
<td style="text-align: center; vertical-align: middle;">10</td>
<td style="text-align: center; vertical-align: middle;">25</td>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">OFG1100</td>
<td style="text-align: center; vertical-align: middle;">5</td>
<td style="text-align: center; vertical-align: middle;">35</td>
</tr>
<tr>
<td style="text-align: center; vertical-align: middle;">OFG1200</td>
<td style="text-align: center; vertical-align: middle;">5</td>
<td style="text-align: center; vertical-align: middle;">28</td>
</tr>
</table>


El método `find()` devuelve el primer elemento encontrado. Para escanear todo el documento y extraer más de un elemento (por ejemplo, si en la página existieran varias tablas), se utiliza el método `find_all()`.

Ver: https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=find#find

Una vez extraída la tabla, usamos `find_all()` para encontrar todas las filas de la tabla (`'tr'`: table rows), e iteramos sobre las filas. Para cada fila, encontramos todos los campos de datos (`'td'`: table data), y vamos agregando los datos a un DataFrame especialmente creado para este fin.

In [6]:
import pandas as pd

#creamos un DataFrame vacío con la estructura de columnas deseada. Lo completaremos con datos extraidos de la tabla.
df=pd.DataFrame(columns=['Curso','Creditos','nEstudiantes'])

#Buscar la tabla con datos
table=soup.find('table')

#Buscar todas las filas de la tabla
rows=table.find_all('tr')

#Iteramos para cada fila (excepto la primera que corresponde a los encabezados)

for row in rows[1::]:
    
    #Buscar todos los campos de datos o columnas de la fila
    cols = row.find_all("td")
    
    #Para cada columna, extraigo el texto
    col_text=[c.text for c in cols]
    
    #Agrego los datos como una nueva fila en el dataframe df
    df=df.append({'Curso':col_text[0],'Creditos':col_text[1],'nEstudiantes':col_text[2]},ignore_index=True)
df

/var/folders/5x/tlp5fd614dv9y_wn8nmtqv4rqq_frn/T/ipykernel_24896/412666707.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'Curso':col_text[0],'Creditos':col_text[1],'nEstudiantes':col_text[2]},ignore_index=True)
/var/folders/5x/tlp5fd614dv9y_wn8nmtqv4rqq_frn/T/ipykernel_24896/412666707.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'Curso':col_text[0],'Creditos':col_text[1],'nEstudiantes':col_text[2]},ignore_index=True)
/var/folders/5x/tlp5fd614dv9y_wn8nmtqv4rqq_frn/T/ipykernel_24896/412666707.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'Curso':col_text[0],'Creditos':col_text[1],'nEstudiantes':col_text[2]},ignore_index=True)
/var/folders/5x/tlp5fd614dv9y_wn8nmtq

,Curso,Creditos,nEstudiantes
0,IMT2100,10,42
1,IMT2200,10,25
2,OFG1100,5,35
3,OFG1200,5,28
